In [707]:
"""
Скрипт для создания сводный таблиц по отчету
"""

'\nСкрипт для создания сводный таблиц по отчету\n'

In [708]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
import numpy as np
import re

In [709]:
def count_uniq(value):
    ser_stat = value.describe() # плоучаем статистику
    return ser_stat.iloc[1] # возвращаем количество уникальных значений


    


In [710]:
def find_top_value(value):
    ser_stat = value.describe() # плоучаем статистику
    return ser_stat.iloc[2] # возвращаем самое частое значение

In [711]:
def count_top_value(value):
    ser_stat = value.describe() # плоучаем статистику
    return ser_stat.iloc[3] # возвращаем количество самых частых значений 

In [712]:
def count_dupl_value(value):
    lst_dupl = [value for value in value.duplicated(keep=False).tolist() if value]
    return len(lst_dupl) # количество дубликатов


In [713]:
str_column ='20,22' # колонки для сводной таблицы
#str_column = 'fgg'
str_target_column = '2,6' # целевая колонка

In [714]:
file_data = 'data/Сводная таблица/data.xlsx'
sheet_name = 'Лист1'
end_folder = 'data/Сводная таблица/result'

In [715]:
base_df = pd.read_excel(file_data,sheet_name=sheet_name)

In [716]:
# обрабатываем список колонок по которым нужно группировать
_lst_cols = re.findall(r'\d+',str_column) # находим цифры
if _lst_cols:
    lst_number_cols =  list(map(int,_lst_cols))
else:
    print('Введите порядковые номера колонок в виде цифр')

In [717]:
lst_number_cols

[20, 22]

In [718]:
_target_col =re.findall(r'\d+',str_target_column)
if _target_col:
    lst_target_number_column =  list(map(int,_target_col))
else:
    print('Введите порядковые номера колонок в виде цифр')

In [719]:
lst_target_number_column

[2, 6]

In [720]:
lst_name_column = [] # список для хранения названий колонок сводной таблицы
for value in lst_number_cols:
    if value != 0:
        lst_name_column.append(df.columns[value-1])
        

In [721]:
lst_name_column

['Образование', 'Группа']

In [722]:
lst_target_name_column = [] # список для хранения названий колонок сводной таблицы
for value in lst_target_number_column:
    if value != 0:
        lst_target_name_column.append(df.columns[value-1])

In [723]:
lst_target_name_column

['Фамилия', 'Средний балл']

In [724]:

dct_func = {'Сумма':'sum','Среднее':'mean','Медиана':'median','Минимум':'min','Максимум':'max','Количество':'count',
           'Количество уникальных':count_uniq,'Самое частое':find_top_value,'Количество самых частых':count_top_value,
           'Количество дубликатов':count_dupl_value}

set_text_func = {'Количество уникальных','Самое частое','Количество самых частых','Количество дубликатов'}

In [725]:
wb = openpyxl.Workbook() # создаем объект
for idx,items in enumerate(dct_func.items()):
    name_sheet,name_func = items # распаковываем
    wb.create_sheet(title=name_sheet,index=idx) # создаем лист
    df = base_df.copy() # копируем дотафрейм что избежать косяков с изменением значений
    if name_sheet not in set_text_func:   
    
        df[lst_target_name_column] = df[lst_target_name_column].applymap(lambda x: x if isinstance(x,(int,float)) else 0)

        temp_df =pd.pivot_table(df,
                  index=lst_name_column,
                  values=lst_target_name_column,             
                  aggfunc=name_func,
                               fill_value=0)
        temp_df = temp_df.reset_index()
        


        for row in dataframe_to_rows(temp_df,index=False,header=True):
            wb[name_sheet].append(row)
        for column in wb[name_sheet].columns:
            max_length = 0
            column_name = get_column_letter(column[0].column)
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            wb[name_sheet].column_dimensions[column_name].width = adjusted_width
    else:
        df[lst_target_name_column] = df[lst_target_name_column].fillna('Не заполнено')
        df[lst_target_name_column] = df[lst_target_name_column].applymap(str)
        temp_df =pd.pivot_table(df,
                  index=lst_name_column,
                  values=lst_target_name_column,             
                  aggfunc=name_func,
                               )
        temp_df = temp_df.reset_index()


        for row in dataframe_to_rows(temp_df,index=False,header=True):
            wb[name_sheet].append(row)
        for column in wb[name_sheet].columns:
            max_length = 0
            column_name = get_column_letter(column[0].column)
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2)
            wb[name_sheet].column_dimensions[column_name].width = adjusted_width
        
del wb['Sheet']
    



In [726]:
wb.sheetnames

['Сумма',
 'Среднее',
 'Медиана',
 'Минимум',
 'Максимум',
 'Количество',
 'Количество уникальных',
 'Самое частое',
 'Количество самых частых',
 'Количество дубликатов']

In [727]:
wb.save(f'{end_folder}/Тест.xlsx')

In [728]:
c = df['ИНН'].describe()

In [729]:
c

count             1001
unique            1000
top       806441417954
freq                 2
Name: ИНН, dtype: object

In [730]:
c.iloc[1]

1000

In [731]:
c.iloc[2]

'806441417954'

In [732]:
pd.pivot_table(df,
              index=lst_name_column,
              values=['Фамилия','Направление_подготовки'],             
              aggfunc=count_uniq,
                           fill_value=0)

Направление_подготовки  Фамилия
Образование              Группа                                 
Основное общее           АГ-596                       2       20
                         АД-393                       1       20
                         АЛ-480                       1       20
                         АП-380                       2       20
                         АЯ-322                       1       20
                         АЯ-849                       7       20
                         БВ-523                       2       20
                         БЕ-281                      10       20
                         БЕ-944                       1       19
                         БН-321                       8       20
                         БЯ-877                       1       20
                         БЯ-928                      10       20
                         ВД-204                      10       19
                         ВЕ-129                       8       20
                         ГВ-833                       2       20
                         ГГ-759                      10       20
                         ДА-989                       7       15
                         ДВ-244                      11       20
                         ДО-951                       1       20
                         ДП-786                       8       20
                         ЕА-479                       1       20
                         ЕД-195                       1       20
                         ЕЕ-374                       1       20
                         ЕП-951                       1       20
                         КА-426                      10       20
                         КА-733                       1       20
                         КВ-838                       1       20
                         КД-856                       1       20
                         КО-589                       1       20
                         ЛЛ-514                       1       20
                         ЛН-300                      10       20
                         ЛО-368                       1       20
                         ЛП-187                       1       20
                         НБ-368                       1       20
                         НЕ-643                       1       20
                         НО-608                       9       20
                         НП-652                       2       20
                         НЭ-787                       9       20
                         НЯ-427                       8       20
                         ОБ-280                       1       20
                         ОВ-173                       8       20
                         ОГ-541                       8       20
                         ПН-298                       1       20
                         ПН-962                       2       20
                         ПЭ-344                      10       19
                         ЭД-420                       1       20
                         ЭН-535                       1       20
                         ЯА-476                       8       20
                         ЯК-419                      10       20
                         ЯЯ-331                       2       20
Среднее общее            ДА-989                       3        4
Среднее профессиональное ДА-989                       1        1
                         ПЭ-344                       1        1